# Stores Locations Installation Problem 

In [1]:
from amplpy import AMPL, Environment
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Ask for the AMPL folder path
amplUserFolder = input("path of the AMPL folder: ")

In [ ]:
# Ask for the input data file
inputFilePath = input("path of the input file (.dat): ")

In [4]:
# Create an AMPL instance
ampl = AMPL(Environment(amplUserFolder))

# Interpret the two files
ampl.read('minimart_sol.mod')
ampl.read_data(inputFilePath)

In [5]:
# Solve the store locations installation problem 
ampl.set_option('solver', amplUserFolder+"/cplex")
ampl.solve()

CPLEX 20.1.0.0: optimal integer solution; objective 27
0 MIP simplex iterations
0 branch-and-bound nodes


In [6]:
totalcost = ampl.get_objective('cost')
# Print it
print("Objective is:", totalcost.value())

Objective is: 27.0


In [7]:
stores = ampl.get_variable('y').get_values().toDict()

# Installed stores coordinates          
storesCoordX = ampl.get_parameter('Cx').get_values().toDict()
storesCoordY = ampl.get_parameter('Cy').get_values().toDict()
storesCoords = list()

storesIndxs = list()

for i in range(1, len(stores)+1):
    if stores.get(i) == 1:
        storesCoords.append((storesCoordX.get(i), storesCoordY.get(i)))
        storesIndxs.append(i)
        
storesCoords

[(22.0, 5.0), (8.0, 17.0), (34.0, 25.0), (40.0, 39.0)]

# Refurbishing Routes Problem

In [8]:
distances = ampl.get_parameter('distance').get_values().toDict()
distances

{(1.0, 1.0): 0.0,
 (1.0, 2.0): 21.400934559032695,
 (1.0, 3.0): 5.0990195135927845,
 (1.0, 4.0): 7.0,
 (1.0, 5.0): 13.038404810405298,
 (1.0, 6.0): 7.280109889280518,
 (1.0, 7.0): 17.46424919657298,
 (1.0, 8.0): 11.180339887498949,
 (1.0, 9.0): 20.8806130178211,
 (1.0, 10.0): 6.4031242374328485,
 (1.0, 11.0): 32.01562118716424,
 (1.0, 12.0): 25.179356624028344,
 (1.0, 13.0): 24.758836806279895,
 (1.0, 14.0): 38.600518131237564,
 (1.0, 15.0): 25.0,
 (1.0, 16.0): 30.610455730027933,
 (1.0, 17.0): 35.510561809129406,
 (1.0, 18.0): 26.0,
 (1.0, 19.0): 26.019223662515376,
 (1.0, 20.0): 35.35533905932738,
 (2.0, 1.0): 21.400934559032695,
 (2.0, 2.0): 0.0,
 (2.0, 3.0): 18.439088914585774,
 (2.0, 4.0): 26.248809496813376,
 (2.0, 5.0): 24.08318915758459,
 (2.0, 6.0): 27.586228448267445,
 (2.0, 7.0): 10.44030650891055,
 (2.0, 8.0): 19.1049731745428,
 (2.0, 9.0): 7.615773105863909,
 (2.0, 10.0): 22.20360331117452,
 (2.0, 11.0): 42.579337712087536,
 (2.0, 12.0): 23.323807579381203,
 (2.0, 13.0): 3

In [9]:
def computeSavings():
    savings = [(distances.get((1,i))+distances.get((1,j))-distances.get((i,j)), i, j) for i in storesIndxs  for j in storesIndxs if i!=j and i>j]
    savings.sort(reverse=True)
    return savings     

In [10]:
computeSavings()

[(45.303149471627904, 20, 12),
 (23.256483603679833, 12, 2),
 (18.285505495017382, 20, 2),
 (8.060865158039704, 3, 2),
 (3.0754351201502423, 12, 3),
 (1.621382895024965, 20, 3)]

## TODO: Clarke-Wright